# AMBER pose estimation steps

This notebook walks you through the pose estimation steps of the AMBER pipeline. 

The steps for pose estiamtion are:
1. Pose estimation for dams for all videos
2. Create videos to check dam tracking
3. Pose estimation for pups for all videos
4. Create videos to check pup detections
5. "Unpickle" pup detection files to convert to csv
6. Join and reformat pup and dam pose estimation output so it is ready to use with SimBA

These steps will be performed for all of the videos in the directory you specify. The provided code will use the example_videos folder in the AMBER-pipeline directory. 

_Be sure to run this in your DeepLabCut conda environment_

In [ ]:
#import required packages and modules
import os
import shutil
import sys
import deeplabcut
import pheno_pickle_raw as PhenoPickleRaw
import join_dam_pup
import pandas as pd
print("Finished importing packages and modules")

Specify the full path to the dam and pup pose estimation config.yaml files. 

The config.yaml can be found in AMBER-pipeline/dam-single-animal-Hannah-2022-05-26 and AMBER-pipeline/pup-nine-pt-Hannah-2022-06-05 directories, but you should update the beginning of the path with the location of the AMBER-pipeline folder on your computer.

In [ ]:
dam_config = './dam-single-animal-Hannah-2022-05-26/config.yaml'
pup_config = './pup-nine-pt-Hannah-2022-06-05/config.yaml'

Specify the path to your directory containing the videos to be analyzed and the type of videos using the video extension. 

In [ ]:
video_directory = './tutorial_videos'
vid_type = '.mp4'

## Dam pose estimation

First, run dam pose estimation on all of the videos in the directory. If you have a lot of videos or your videos are long, this could take a long time.
For more information on the deeplabcut.analyze_videos function, see the [DLC documentation](https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html)

In [ ]:
deeplabcut.analyze_videos(dam_config, [video_directory], save_as_csv = True, videotype=vid_type)
print("Dam pose estimation complete")

### Create dam pose estimation videos

It's important to check that the pose estimation models are performing well with your videos. One way to check this is to create a labeled video. Once the videos are created, they are moved to a new subdirectory called "pose_estimation_videos" so the labeled videos are not used for pup pose estimation in the next steps. 

In [ ]:
deeplabcut.create_labeled_video(dam_config, [video_directory], save_frames=False)
track_video = video_directory + os.sep + 'pose_estimation_videos' + os.sep
  
try:
    os.makedirs(track_video)
except FileExistsError:
    pass

for file_name in os.listdir(video_directory):
    if "dam-single-animal" in file_name and file_name.endswith('.mp4'):
        src_path = os.path.join(video_directory, file_name)
        dst_path = os.path.join(track_video, file_name)
        shutil.move(src_path, dst_path)
print("Completed video creation for dam labels")

## Pup pose estimation

Now run pup pose estimation on the videos. The auto_track argument is set to false to only run the detection portion of the multi-animal workflow. This avoids the loss in pup key point detections during individual assembly (when key points are assigned to indidvudal pups during the second part of the DLC multi-animal workflow). See the [DLC docs](https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html) for more information.

In [ ]:
deeplabcut.analyze_videos(pup_config, [video_directory], auto_track=False)
print("Finished pup pose estimation")

### Create pup detections videos

After labeled videos with pup detections are created, they are also moved to the "pose_estimation_videos" folder. Check the videos after they are created to make sure the tracking looks good.  

In [ ]:
deeplabcut.create_video_with_all_detections(pup_config, [video_directory])

for file_name in os.listdir(video_directory):
    if "pup-nine-pt" in file_name and file_name.endswith('.mp4'):
        src_path = os.path.join(video_directory, file_name)
        dst_path = os.path.join(track_video, file_name)
        shutil.move(src_path, dst_path)

print("Finished creating pup labeled videos")

### Convert pup detection file from .pickle to .csv

The code below will use the pheno_pickle_raw.py script to convert the .pickle files created in the step above that contain the pup detections to csv files. 

Note: Although detected points are placed in columns contianing pup IDs, this placement is arbitrary and does not necessarily mean that the detected point belongs to that particular pup. For exmaple, the nose, eye, head, and back points assigned to 'pup1' may belong to the same individual pup or may belong to different pups. Likewise, key point assignments to individual pups are not consistent across frames. For example, the nose point assinged to "pup1" in frame n and n+1 may belong to different individuals. The behavior classifiers consider the all the pups in the litter as a unit, so it is not necessary to know the idenities of individuals pups within or across frames. 

If you are interested tracking individual pups, you will need to convert detections to tracklets, stitch tracklets, and (optionally) refine tracklets using DeepLabCut. 


In [ ]:
pickle_dir = video_directory.replace("'", "")
PhenoPickleArgs = ['x', '-input_directory:' + pickle_dir]
PhenoPickleRaw.main(PhenoPickleArgs)

# Join dam and pup pose estimation data

Now that you have csv files for dam and pup tracking infomration for all of the videos, it can be combined and reformatted so it is ready for behavior classification in SimBA. to do this, run the join_dam_pup.py script. This script: 
1. Identifies CSV files for dam and pup videos based on the original video file name.
2. Loads and preprocesses the data from these files.
3. Merges the dam and pup data based on frame timestamps.
4. Saves the merged data to CSV files in a subdirectory named 'AMBER_joined_pose_estimation'.

In [ ]:
def main(argv):
    print("Starting file joining...")
    all_files = os.listdir(video_directory)
    
    #given the video directory, find all files containing "DLC_resnet50_dam-single-animalMay26shuffle" and ending in ".csv"
    dam_files = []
    dam_keys = []
    for file in all_files:
        if 'DLC_resnet50_dam-single-animalMay26shuffle' in file \
                and file.lower().endswith('.csv'):
            dam_files.append(video_directory + os.sep + file)
            dam_keys.append(file.split('DLC_resnet50_dam-single-animalMay26shuffle')[0])
    #print("Dam files:", dam_files)

    #given the video directory, find all files containing "DLC_dlcrnetms5_pup-nine-ptJun5shuffle" and ending in "UNPICKLED.csv"
    pup_files = []
    pup_keys= []
    for file in all_files:
        if 'DLC_dlcrnetms5_pup-nine-ptJun5shuffle' in file \
                and file.lower().endswith('unpickled.csv'):
            pup_files.append(video_directory + os.sep + file)
            pup_keys.append(file.split('DLC_dlcrnetms5_pup-nine-ptJun5shuffle')[0])
    #print("Pup files:", pup_files)

    #create paired_keys for files that have pup and dam pose estimation
    dam_set = set(dam_keys)
    pup_set = set(pup_keys)
    paired_keys = list(dam_set.intersection(pup_set))
    #print("Paired keys:", paired_keys)

    for key in paired_keys:
        print('Joining', key)
        for dam_file in dam_files:
            if video_directory + os.sep + key + 'DLC_resnet50_dam-single-animalMay26shuffle' in dam_file:
                dam_path = dam_file
                break

        dam_df = pd.read_csv(dam_path)
        dam_df.columns = dam_df.iloc[0] + '_' + dam_df.iloc[1]
        dam_df = dam_df.iloc[2:]
        dam_df = dam_df.rename(columns= {'bodyparts_coords': 'frame'})
        dam_df['frame'] =dam_df['frame'].astype(int)

        for pup_file in pup_files:
            if video_directory + os.sep + key + 'DLC_dlcrnetms5_pup-nine-ptJun5shuffle' in pup_file:
                pup_path = pup_file
                break

        pup_df = pd.read_csv(pup_path)
        print("pre merge")
        merged_df = dam_df.merge(pup_df, on='frame', how='left')
        merged_df.loc[-2] = [column_name.replace('_x', '').replace('_y', '').replace('_likeihood','') for column_name in merged_df.columns]
        merged_df.loc[-1] = ['coords'] + ['x', 'y', 'likelihood'] * int((len(merged_df.columns) - 1) / 3)
        merged_df.index = merged_df.index + 2
        merged_df.sort_index(inplace=True)
        merged_df.columns = ['scorer'] + (['DLC_AMBER_dam_pup'] * (len(merged_df.columns) - 1))
        merged_df.iloc[0, 0] = 'bodyparts'
        print("post merge")

        try:
            os.makedirs(video_directory + os.sep + 'AMBER_joined_pose_estimation' + os.sep)
        except FileExistsError:
            pass
        out_path = video_directory + os.sep + 'AMBER_joined_pose_estimation' + os.sep + key + '.csv'
        merged_df.to_csv(out_path, index=False)
        print(out_path)


main(video_directory)

print("Finished joining dam and pup files")